# DataLoader

In [41]:
# DO NOT RUN THIS CELL..WORK IN PROGRESS

from __future__ import absolute_import, division, print_function

import os
import random
import numpy as np
import copy
from PIL import Image  # using pillow-simd for increased speed

import torch
import torch.utils.data as data
from torchvision import transforms

In [42]:
# DO NOT RUN THIS CELL..WORK IN PROGRESS

def pil_loader(path):
    # open path as file to avoid ResourceWarning
    # (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [43]:
# DO NOT RUN THIS CELL..WORK IN PROGRESS

class MonoDataset(data.Dataset):
    
    def __init__(self,data_path,filenames,height,width,frame_idxs,num_scales,is_train=False,img_ext='.jpg'):
        super(MonoDataset, self).__init__()
        
        self.data_path = data_path
        self.filenames = filenames
        self.height = height
        self.width = width
        self.num_scales = num_scales
        self.frame_idxs = frame_idxs
        self.is_train = is_train
        self.img_ext = img_ext
        
        self.interp = Image.ANTIALIAS
        self.loader = pil_loader
        self.to_tensor = transforms.ToTensor()
        
        
    def __len__(self):
        return len(self.filenames)
    

In [22]:
import datasets
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

In [30]:
#dataset Parameters
data_path = "/home/ubuntu/monodepth2/kitti_data"
height = 192
width = 640
frame_ids = [0, -1, 1]
num_scales = 4
img_ext = '.png'

#Loader Parameters
batch_size = 12
num_workers = 12

In [31]:
fpath_train = "/home/ubuntu/monodepth2/splits/eigen_zhou/train_files.txt"
f_train = open(fpath_train)
train_filenames = f_train.readlines()
#train_filenames

In [32]:
fpath_val = "/home/ubuntu/monodepth2/splits/eigen_zhou/val_files.txt"
f_val = open(fpath_val)
val_filenames = f_val.readlines()
#val_filenames

In [36]:
datasets_dict = {"kitti": datasets.KITTIRAWDataset}
dataset = datasets_dict["kitti"]

In [37]:
#Train
train_dataset = dataset(data_path, train_filenames, height, width,frame_ids,num_scales,is_train=True, img_ext=img_ext)
train_loader = DataLoader(train_dataset, batch_size, True,num_workers=num_workers, pin_memory=True, drop_last=True)

In [38]:
#Validation
val_dataset = dataset(data_path, val_filenames, height, width,frame_ids, 4, is_train=False, img_ext=img_ext)
val_loader = DataLoader(val_dataset, batch_size, True,num_workers=num_workers, pin_memory=True, drop_last=True)